In [1]:
import pandas as pd
import numpy as np
import csv
import math

In [ ]:
input_csv = input()
out_csv = input()
feature_count = 3
new_train = pd.DataFrame()

model = np.load('model.npy')

topic = ['id','測項']+[i for i in range(9)]
test = pd.read_csv(input_csv,names = topic)
for i in test:
    test[i].replace({'NR':0.0},inplace = True)

PM2_5_t = test[test['測項']=='PM2.5'].iloc[:,2:]
CO_t = test[test['測項']=='CO'].iloc[:,2:]
PM10_t = test[test['測項']=='PM10'].iloc[:,2:]
test = pd.DataFrame(pd.concat([CO_t,PM10_t,PM2_5_t]).sort_index().values).apply(pd.to_numeric, errors='ignore')

x_test = []
for j in range(0,test.shape[0],feature_count):
    count = test.shape[1]-9
    if count > 0:
        for i in range(test.shape[1]-9):
            x_t = np.array(test.iloc[j:j+feature_count,i+2:i+9])
            x_test.append(x_t.flatten())
    else:
        x_t = np.array(test.iloc[j:j+feature_count,2:])
        x_test.append(x_t.flatten())
X_t = np.array(x_test,float)

X_t=(X_t - model[2]) / (model[1] - model[2])
X_test = np.insert(X_t,0,1,axis = 1)
Y_test = np.dot(X_test,model[0])
id_ = np.array([])
for i in range(len(Y_test)):
    id_=np.append(id_,'id_'+str(i))
final = pd.DataFrame(np.concatenate(([id_],[Y_test])).transpose(),columns = ['id','value'])
final.to_csv(out_csv,index = False)